#### List of Frames
- stock_df (base frame)
- crypto_df (base frame)
- companies_df (base frame)



In [1]:
# Import Modules

import pandas as pd
from datetime import timedelta


In [2]:
# Load Stock & Crypto Dataframes

stock_csv = "stocks.csv"
stock_df = pd.read_csv(stock_csv)

crypto_csv = "cryptocurrency.csv"
crypto_df = pd.read_csv(crypto_csv)

companies_csv = "companies.csv"
companies_df = pd.read_csv(companies_csv)

In [3]:
# Cleaning Stock Data

stock_df = stock_df.replace({'\%': ''}, regex=True)
stock_df = stock_df.replace({'\+': ''}, regex=True)
stock_df = stock_df.rename(columns = {'last': 'price_usd'})

def convert_volume(value):
    """Convert volume strings with K, M, B suffixes to numbers"""
    if pd.isna(value):
        return value
    
    value = str(value).strip()
    
    if value.endswith('K'):
        return float(value[:-1]) * 1_000
    elif value.endswith('M'):
        return float(value[:-1]) * 1_000_000
    elif value.endswith('B'):
        return float(value[:-1]) * 1_000_000_000
    else:
        return float(value)

stock_df['vol_'] = stock_df['vol_'].apply(convert_volume)
stock_df = stock_df.drop('ticker', axis=1, errors='ignore')

lookup = companies_df[['ticker', 'company name', 'short name']].copy()

lookup['company_lower'] = lookup['company name'].str.lower().str.strip()
lookup['short_lower'] = lookup['short name'].str.lower().str.strip()

stock_df['name_lower'] = stock_df['name'].str.lower().str.strip()

stock_df = stock_df.merge(lookup[['ticker', 'company_lower']], 
                          left_on='name_lower', 
                          right_on='company_lower', 
                          how='left', 
                          suffixes=('', '_drop'))

mask = stock_df['ticker'].isna()
temp_merge = stock_df[mask].drop('ticker', axis=1).merge(
    lookup[['ticker', 'short_lower']], 
    left_on='name_lower', 
    right_on='short_lower', 
    how='left'
)
stock_df.loc[mask, 'ticker'] = temp_merge['ticker'].values

stock_df = stock_df.drop(['name_lower', 'company_lower', 'short_lower'], axis=1, errors='ignore')

cols = stock_df.columns.tolist()
name_idx = cols.index('name')
ticker_idx = cols.index('ticker')
cols.insert(name_idx + 1, cols.pop(ticker_idx))
stock_df = stock_df[cols]

stock_df = stock_df.rename(columns = {'ticker': 'symbol'})

In [4]:
# Cleaning Crypto Data

crypto_df = crypto_df.replace({'\$': ''}, regex=True)
crypto_df = crypto_df.replace({'\%': ''}, regex=True)
crypto_df = crypto_df.replace({'\+': ''}, regex=True)

In [5]:
# Cleaning Company Data

companies_df = companies_df.drop('logo', axis=1)
companies_df = companies_df.drop('market cap', axis=1)
companies_df = companies_df.drop('ceo', axis=1)
companies_df = companies_df.drop('website', axis=1)
companies_df = companies_df.drop('description', axis=1)

In [6]:
# Stock Base Frame 
stock_df.head(3)

,timestamp,name,symbol,price_usd,high,low,chg_,chg_%,vol_,time
0,2025-10-14 18:00:03,NVIDIA,NVDA,181.19,184.80,179.70,-7.13,-3.79,74860000.0,10:57:55
1,2025-10-14 18:00:03,Amazon.com,AMZN,216.19,216.92,212.60,-3.88,-1.76,16800000.0,10:58:30
2,2025-10-14 18:00:03,3M,NaN,151.84,152.19,149.28,0.74,0.49,479840.0,10:58:34


In [7]:
# Crypto Base Frame
crypto_df.head(3)

,timestamp,name,symbol,price_usd,vol_24h,total_vol,chg_24h,chg_7d,market_cap
0,2025-10-14 18:42:04,Wrapped stETH,wsteth,"4,862.78","17,303,242.00",0.00,-3.08,-13.06,"15,541,897,587"
1,2025-10-14 18:42:04,Figure Heloc,figr_heloc,1.00,"5,942,967.00",0.00,2.52,0.17,"12,973,755,429"
2,2025-10-14 18:42:04,Wrapped eETH,weeth,"4,312.52","13,561,342.00",0.00,-3.06,-13.12,"10,582,898,874"


In [8]:
# Companies Base Frame
companies_df.head(3)

,ticker,company name,short name,industry,exchange,sector,tag 1,tag 2,tag 3
0,A,Agilent Technologies Inc.,Agilent,Medical Diagnostics & Research,New York Stock Exchange,Healthcare,Healthcare,Diagnostics & Research,Medical Diagnostics & Research
1,AA,Alcoa Corporation,Alcoa,Metals & Mining,New York Stock Exchange,Basic Materials,Basic Materials,Aluminum,Metals & Mining
2,AABA,Altaba Inc.,Altaba,Asset Management,Nasdaq Global Select,Financial Services,Financial Services,Asset Management,NaN
